In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from pathlib import Path
import numpy as np
import PIL
import PIL.Image
import json

#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}");


2023-10-05 18:24:23.100050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-05 18:24:23.100083: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-05 18:24:23.100110: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.14.0
len(devices):  0
available GPUs: []


2023-10-05 18:24:25.841873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


**Dataloaders**

In [2]:
import data_setup

# try to import the dataset
dataset="SkyData"
BATCH_SIZE = 2
SHUFFLE_BUFFER_SIZE = 100

train_dataloader,test_dataloader = data_setup.create_dataloaders(dataset=dataset, 
                                                                BATCH_SIZE=BATCH_SIZE,
                                                                SHUFFLE_BUFFER_SIZE=100
                                                                )
#

[INFO] loading train dataset
[INFO] train _dataset:  27700
[INFO] loading val dataset
[INFO] val _dataset:  7990


## **Model**

In [3]:
import model_setup
import Utils
rgb_inputs_shape = (192,192,3)
ir_inputs_shape =  (128,128,3)
output_channels_per_block = 64
regression_input_shape = (*rgb_inputs_shape[:2],output_channels_per_block*2)
regression_output_shape = 8


In [4]:
featureEmbeddingBackBone = model_setup.getFeatureEmbeddingBackBone(rgb_inputs_shape=rgb_inputs_shape,
                                                        ir_inputs_shape=ir_inputs_shape,
                                                        output_channels_per_block=output_channels_per_block
                                                        )

regressionHead= model_setup.getRegressionHead(input_shape=regression_input_shape,
                                                output_size=regression_output_shape
                                                )


**Visualize and save model structures**

In [5]:
# # visualize and save models

# Utils.plot_and_save_model_structure(featureEmbeddingBackBone,
#                                             save_path="resources/",
#                                             save_as=f"featureEmbeddingBackBone")
# Utils.plot_and_save_model_structure(regressionHead,
#                                             save_path="resources/",
#                                             save_as=f"regressionHead")

## **Training**


In [6]:
import engine 

initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                                decay_steps=10000,
                                                                decay_rate=0.96,
                                                                staircase=True)
NUM_EPOCHS = 1

# Setup optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Start the timer
from timeit import default_timer as timer
start_time = timer()
# Train model 
model_results = engine.train_first_stage(model=featureEmbeddingBackBone,
                                                train_dataloader=train_dataloader,
                                                test_dataloader=test_dataloader,
                                                optimizer=optimizer,
                                                epochs=NUM_EPOCHS)
# End the timer and print out how long it took
end_time = timer()
print(f"Total training time : {end_time-start_time:.3f} seconds")

/home/nalain/miniconda3/envs/VisIrNet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] TrainingfeatureEmbeddingBackbone for 1 epochs


  0%|          | 0/1 [00:01<?, ?it/s]


InvalidArgumentError: {{function_node __wrapped__RealDiv_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [2,3,3] vs. [2] [Op:RealDiv] name: 